# Experimento 1
Usando la red de spiking_jelly, es decir, DVSGesture_net probar su rendimiento en distintos datasets.


In [1]:
import torch
cuda_available = torch.cuda.is_available()
print(cuda_available)

True


In [1]:
import Laboratory

/Users/marcosesquivelgonzalez/pyenviroments/tfm/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## DVSAnimals dataset

In [2]:
data_dvsgesture = '/Users/marcosesquivelgonzalez/Desktop/MasterCDatos/TFM/data/DVS_Gesture_dataset'
Laboratory.execute_experiment(inp_data=data_dvsgesture)

ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


The directory [/Users/marcosesquivelgonzalez/Desktop/MasterCDatos/TFM/Datasets/DVS_Gesture_dataset/frames_number_16_split_by_number] already exists.
The directory [/Users/marcosesquivelgonzalez/Desktop/MasterCDatos/TFM/Datasets/DVS_Gesture_dataset/frames_number_16_split_by_number] already exists.


wandb: Currently logged in as: marcosesgonz (tfm_datcom). Use `wandb login --relogin` to force relogin


Abortar escribiendo exit


Instancias de train:  1176
Instancias de test:  288
Mkdir /Users/marcosesquivelgonzalez/Desktop/MasterCDatos/TFM/Projects/result_logs/DVS_Gesture_dataset/T16_b8_lr0.1.
/Users/marcosesquivelgonzalez/Desktop/MasterCDatos/TFM/Projects/result_logs/DVS_Gesture_dataset/T16_b8_lr0.1
epoch = 0, train_loss = 0.0909, train_acc = 0.0825, test_loss = 0.0909, test_acc = 0.0833, max_test_acc = 0.0833
train speed = 3.8811 images/s, test speed = 7.8287 images/s
escape time = 2023-06-30 23:45:02



In [2]:
data_animals = '/Users/marcosesquivelgonzalez/Desktop/MasterCDatos/TFM/data/SLAnimals_Dataset'
Laboratory.execute_experiment(inp_data=data_animals)

Starting new version
The directory [/Users/marcosesquivelgonzalez/Desktop/MasterCDatos/TFM/Datasets/SLAnimals_Dataset/frames_number_16_split_by_number] already exists.
Posible labels: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18]


ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: marcosesgonz (tfm_datcom). Use `wandb login --relogin` to force relogin


Abortar escribiendo exit


Instancias de train:  896
Instancias de test:  225
/Users/marcosesquivelgonzalez/Desktop/MasterCDatos/TFM/Projects/result_logs/SLAnimals_Dataset/T16_b8_lr0.1
epoch = 0, train_loss = 0.0526, train_acc = 0.0525, test_loss = 0.0526, test_acc = 0.0533, max_test_acc = 0.0533
train speed = 3.4798 images/s, test speed = 5.8741 images/s
escape time = 2023-06-30 22:55:10

/Users/marcosesquivelgonzalez/Desktop/MasterCDatos/TFM/Projects/result_logs/SLAnimals_Dataset/T16_b8_lr0.1
epoch = 1, train_loss = 0.0526, train_acc = 0.0525, test_loss = 0.0526, test_acc = 0.0533, max_test_acc = 0.0533
train speed = 3.5608 images/s, test speed = 6.0359 images/s
escape time = 2023-06-30 22:51:44

/Users/marcosesquivelgonzalez/Desktop/MasterCDatos/TFM/Projects/result_logs/SLAnimals_Dataset/T16_b8_lr0.1
epoch = 2, train_loss = 0.0526, train_acc = 0.0525, test_loss = 0.0526, test_acc = 0.0533, max_test_acc = 0.0533
train speed = 3.5916 images/s, test speed = 6.0760 images/s
escape time = 2023-06-30 22:50:33

/Use

test_acc,▁▁▁▁▁
test_loss,▁▁▁▁▁
train_acc,▁▁▁▁▁
train_loss,▁▁▁▁▁
test_acc,0.05333
test_loss,0.05263
train_acc,0.05246
train_loss,0.05263


KeyboardInterrupt: 

In [1]:
from spikingjelly.activation_based.model import parametric_lif_net
from spikingjelly.activation_based import functional, surrogate, neuron
import torch

DVS_net = parametric_lif_net.DVSGestureNet(channels=128, spiking_neuron=neuron.LIFNode, surrogate_function=surrogate.ATan(), detach_reset=True)
functional.set_step_mode(DVS_net, 'm')

x = torch.rand(size=[10, 8, 2, 128, 128])
print(DVS_net(x).shape)

torch.Size([10, 8, 22])
